In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import numpy as np
import pygritbx as pgt

In [2]:
# Unit Vectors for defining reference frame
i = np.array([1, 0, 0])
j = np.array([0, 1, 0])
k = np.array([0, 0, 1])
referenceFrame = np.array([i, j, k])

In [3]:
# Shaft A1 Components' Locations
a1 = 46.75 # [mm]
b1 = 43.5 # [mm]

zA = 0 # [mm]
zB = a1
zR1 = a1 + b1 # [mm]

In [4]:
# Spindle P Components' Locations
zR2 = 0 # [mm]
zR3 = 55 # [mm]

In [5]:
# Shaft A2 Components' Locations
a2 = 23.75 # [mm]
b2 = 58 # [mm]
c2 = 77.5 # [mm]

zR4 = 26 # [mm]
zC = zR4 + a2
zD = zC + b2
zOut = zD + c2 # [mm]


In [6]:
# Input
motor = pgt.Motor(name="M1", loc=0, power=6e3, n=2750, axis=k)

In [ ]:
# Material
shaftMaterial = pgt.Material(name='34NiCrMo6', sigma_u=1050, sigma_y=950, sigma_Dm1=520)
R1R4_material = pgt.Material(name='Steel', sigma_u=1060, sigma_y=950, HB=335)
R2R3_material = pgt.Material(name='Steel', sigma_u=1950, sigma_y=1400, HB=560)

In [8]:
# Gears
# R1
R1 = pgt.Gear(name="R1", axis=k, loc=zR1, m_n=2.5, z=18, psi=20, phi_n=20, Q_v=7, FW=30, material=R1R4_material)

# R2
R2 = pgt.Gear(name="R2", axis=k, loc=zR2, m_n=2.5, z=73, psi=-20, phi_n=20, Q_v=7, FW=30, material=R2R3_material)

# R3
R3 = pgt.Gear(name="R3", axis=k, loc=zR3, m_n=2.5, z=19, psi=-20, phi_n=20, Q_v=7, FW=30, material=R2R3_material)

# R4
R4 = pgt.Gear(name="R4", axis=k, loc=zR4, m_n=2.5, z=51, psi=20, phi_n=20, Q_v=7, FW=30, material=R1R4_material)

In [9]:
# Supports
# A (roller 30203)
A = pgt.Support(name="A", type="Roller", bearingType="Tapered", catalogueName="30203", catalogueType="Explorer",
                     d=17, D=40, B=13.25, C=23.4e3, C0=18.6e3, Pu=1.83e3, nr=15e3, a=9, e=0.35, X=0, Y=1.7, Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=-1, arr="B2B", axis=k, loc=0)

# B (roller 30203)
B = pgt.Support(name="B", type="Pin", bearingType="Tapered", catalogueName="30203", catalogueType="Explorer",
                     d=17, D=40, B=13.25, C=23.4e3, C0=18.6e3, Pu=1.83e3, nr=15e3, a=9, e=0.35, X=0, Y=1.7,Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=1, arr="B2B", axis=k, loc=a1)

# C (roller 30208)
C = pgt.Support(name="C", type="Roller", bearingType="Tapered", catalogueName="30208", catalogueType="Explorer",
                     d=40, D=80, B=19.75, C=75.8e3, C0=68e3, Pu=7.65e3, nr=7e3, a=16, e=0.37, X=0, Y=1.6, Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=-1, arr="B2B", axis=k, loc=zR4 + a2)

# D (roller 30208)
D = pgt.Support(name="D", type="Pin", bearingType="Tapered", catalogueName="30208", catalogueType="Explorer",
                     d=40, D=80, B=19.75, C=75.8e3, C0=68e3, Pu=7.65e3, nr=7e3, a=16, e=0.37, X=0, Y=1.6, Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=1, arr="B2B", axis=k, loc=zR4 + a2 + b2)

# F
F = pgt.Support(axis=k)

In [10]:
# Output Component
output = pgt.Component(name="Output", axis=k, loc=zOut)

In [ ]:
# Gearbox Configuration
# Shaft A1 definition
A1 = pgt.Shaft(name="A1", inputs=[motor], outputs=[R1], sups=np.array([A, B]), axis=k, loc=[0, 0, 0], material=shaftMaterial)
# Mesh M1 definition
M1 = pgt.GearMesh(name="M1", drivingGear=R1, drivenGear=R2, radiality=[-j], type="External")
# Spindle P definition
spindleP = pgt.Shaft(name="P", inputs=[R2], outputs=[R3], sups=np.array([F, F]))
# Mesh M2 definition
M2 = pgt.GearMesh(name="M2", drivingGear=R3, drivenGear=R4, radiality=[j], type="External")
A2 = pgt.Shaft(name="A2", inputs=[R4], outputs=[output], sups=np.array([C, D]), axis=k, material=shaftMaterial)

In [ ]:
A1.solve()

In [ ]:
R1.EFs[0].force, R1.ETs[0].torque

In [ ]:
A.F_tot.force, B.F_tot.force

In [ ]:
A1.EFs[0].force, A1.EFs[1].force, A1.EFs[2].force, A1.EFs[3].force

In [ ]:
R1.solve()

In [ ]:
R2.solve()

In [ ]:
spindleP.solve()

In [ ]:
R3.solve()

In [ ]:
R4.solve()

In [ ]:
Fout_r = np.abs(np.sum(R4.ETs[0].torque)) / 0.1 * j
Fout_a = 0.25 * np.abs(np.sum(Fout_r)) * (-k)
Fout = pgt.Force(Fout_a + Fout_r, output.abs_loc)
A2.outputs[0].updateEFs([Fout])

In [ ]:
A2.solve()

In [ ]:
C.F_tot.force, D.F_tot.force

In [ ]:
# Shaft A2: profile for static verification
statProfile = pgt.ShaftProfile(name="A2 Static Profile",
                                  radii=np.array([34, 35, 35, 30, 30, 35, 35, 40, 40, 34, 34, 32, 32, 27, 27, 32, 32, 31]) / 2, 
                                  locs=np.array([0, 1, 11.9, 12, 39.9, 40, 45.9, 46, 132.9, 133, 158.4, 158.5, 178.4, 178.5, 208.4, 208.5, 222, 223]))
statProfile_refined = statProfile.refineProfile(delta=0.1)
statProfile_refined.addFillet(radius=1, quadrant=[2], zOff=45, dOff=18.5)
statProfile_refined.addFillet(radius=1, quadrant=[1], zOff=134, dOff=18)
A2.addProfile(profile=statProfile_refined)

In [ ]:
# Shaft A2: define sections
# v1
v1 = pgt.ShaftSection(name='V1', loc=26, d=30, Ra=0.8, material=shaftMaterial)
v1.appendKf(Kf=np.array([1.6, 2]), loadType=np.array(["Bending", "Torsion"]))

In [ ]:
# v2
v2 = pgt.ShaftSection(name='V2', loc=46, d=40, Ra=0.8, material=shaftMaterial)
v2.addNotchSensitivity(notchRadius=1, sigma_u=A2.material.sigma_u)
v2.addGeometricStressRaiser(r2d=1/35, D2d=40/35)

In [ ]:
# v3
v3 = pgt.ShaftSection(name='V3', loc=133, d=40, Ra=0.8, material=shaftMaterial)
v3.addNotchSensitivity(notchRadius=1, sigma_u=A2.material.sigma_u)
v3.addGeometricStressRaiser(r2d=1/34, D2d=40/34)

In [ ]:
userSections = np.array([v1, v2, v3])
A2.addSections(sections=userSections)

In [ ]:
A2.performStaticVerification(RF=referenceFrame, profile=A2.profiles[0])

In [ ]:
# Shaft A2: profile for fatigue verification
fatigueProfile = pgt.ShaftProfile(name="A2 Fatigue Profile",
                                     radii=np.array([34, 35, 35, 40, 40, 34, 34, 32, 32, 31]) / 2,
                                     locs=np.array([0, 1, 45.9, 46, 132.9, 133, 158.4, 158.5, 222, 223]))

fatigueProfile_refined = fatigueProfile.refineProfile(delta=0.1)
fatigueProfile_refined.addFillet(radius=1, quadrant=[2], zOff=45, dOff=18.5)
fatigueProfile_refined.addFillet(radius=1, quadrant=[1], zOff=134, dOff=18)
A2.addProfile(profile=fatigueProfile_refined)
A2.sections[0].d = 35
userSections[0].d = 35

In [ ]:
A2.performFatigueVerification(RF=referenceFrame, profile=A2.profiles[1])

In [ ]:
R1.analyseGearToothBending(mesh=M1, powerSource="Uniform", drivenMachine="Uniform", dShaft=35, Ce=1, teethCond="uncrowned teeth",
                           lShaft=a1+b1+50, useCond="Commercial, enclosed units", sigma_FP=320, b_YN=1.3558, e_YN=-0.0178,
                           N=1e8, temp=60, rel=0.99)

In [ ]:
R3.analyseGearToothBending(mesh=M2, powerSource="Uniform", drivenMachine="Uniform", dShaft=30, Ce=1, teethCond="uncrowned teeth",
                           lShaft=70, useCond="Commercial, enclosed units", sigma_FP=450, b_YN=1.3558, e_YN=-0.0178,
                           N=1e8, temp=60, rel=0.99)

In [ ]:
R1.analyseGearToothPitting(mesh=M1, Z_R=1, sigma_HP=860, b_ZN=1.4488, e_ZN=-0.023, N=1e8)

In [ ]:
R3.analyseGearToothPitting(mesh=M2, Z_R=1, sigma_HP=1360, b_ZN=1.4488, e_ZN=-0.023, N=1e8)

In [ ]:
# Define oil of choice
oil = pgt.Oil("ISO VG 100", 60, 15, 45)

In [ ]:
A.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=2, oil=oil)

In [ ]:
B.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=5, oil=oil)

In [ ]:
C.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=1, oil=oil)

In [ ]:
D.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=2, oil=oil)